### Convert model to ONNX-format

In [1]:
import torch

print(f'Cuda available: {torch.cuda.is_available()}')

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
print(f'Running on: {device}, Device count: {torch.cuda.device_count()}')

Cuda available: True
Running on: cpu, Device count: 2


### Prepare data sample

In [106]:
data_root = "/home/techtoker/projects/TNT-Trajectory-Predition/dataset/interm_data_small"

In [109]:
from os.path import join as pjoin
from core.trainer.tnt_trainer import TNTTrainer
from core.dataloader.argoverse_loader_low_memory import GraphData, ArgoverseCustom

# Get model input

amount_of_files = 1
test_set = ArgoverseCustom(pjoin(data_root, "{}_intermediate".format('test')), amount_of_files)

In [110]:
sample = test_set[0]
sample.num_graphs = 1
sample.batch = None

In [111]:
torch.save(sample, 'tnt_data_sample.pt')

### Load data sample and model inference

In [2]:
data_sample = torch.load('tnt_data_sample.pt')

In [3]:
from core.model.TNT import TNT

# init or load model
# input dim: (20, 8); output dim: (30, 2)
# model_name = VectorNet
model_name = TNT
model = model_name(
    data_sample.num_features,
    30,
    num_global_graph_layer=1,
    with_aux=True,
    device=device,
    multi_gpu=False
)

# resume from model file or maintain the original
load_path = "/home/techtoker/projects/TNT-Trajectory-Predition/run/tnt/04-06-17-45/final_TNT.pth"

model.load_state_dict(torch.load(load_path, map_location=device))
model.eval()

TNT(
  (criterion): TNTLoss()
  (backbone): VectorNetBackbone(
    (subgraph): SubGraph(
      (layer_seq): Sequential(
        (glp_0): MLP(
          (linear1): Linear(in_features=10, out_features=64, bias=True)
          (linear2): Linear(in_features=64, out_features=64, bias=True)
          (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (act1): ReLU(inplace=True)
          (act2): ReLU(inplace=True)
          (shortcut): Sequential(
            (0): Linear(in_features=10, out_features=64, bias=True)
            (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          )
        )
        (glp_1): MLP(
          (linear1): Linear(in_features=128, out_features=64, bias=True)
          (linear2): Linear(in_features=64, out_features=64, bias=True)
          (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((64,), eps=1e-05, elementwise_affi

In [4]:
out = model.inference(data_sample.to(device)).detach().cpu().numpy()[0]
out

array([[-0.53760064,  0.48874754, -1.352921  ,  1.2777356 , -0.5393727 ,
        -0.93860096, -0.13799694, -0.19833225,  0.49789652,  0.31518498,
        -0.37477338,  0.44033206,  0.30627805,  1.3412883 ,  0.37958625,
         1.5051084 , -0.8344571 ,  1.3999052 , -0.39798447,  0.9200321 ,
         0.3235702 , -1.7662491 ,  0.08818872, -0.46130407, -0.5036478 ,
         1.2338512 ,  0.35978618,  0.19164205,  0.02231276,  0.3069516 ,
        -0.2645611 ,  0.45336705,  0.6222872 ,  0.676536  , -0.2771033 ,
         1.0920799 ,  0.24610464,  0.82191473, -0.51379526,  0.8424963 ,
        -0.17202057,  0.7189366 , -0.10080089, -0.45802483, -0.35277563,
         1.081737  , -0.24125713,  0.39591706,  0.19953634,  1.1147801 ,
         0.00858426,  1.0530682 ,  0.41544342,  0.60319924, -0.42690486,
         0.6458736 ,  0.5402296 ,  1.2734904 , -0.5756928 ,  0.51491797],
       [-0.55162776,  0.5229352 , -1.3283005 ,  1.2402254 , -0.53935814,
        -0.85805094, -0.15366533, -0.12035349,  0.

### Convert to ONNX

In [6]:
model.modules

<bound method Module.modules of TNT(
  (criterion): TNTLoss()
  (backbone): VectorNetBackbone(
    (subgraph): SubGraph(
      (layer_seq): Sequential(
        (glp_0): MLP(
          (linear1): Linear(in_features=10, out_features=64, bias=True)
          (linear2): Linear(in_features=64, out_features=64, bias=True)
          (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (act1): ReLU(inplace=True)
          (act2): ReLU(inplace=True)
          (shortcut): Sequential(
            (0): Linear(in_features=10, out_features=64, bias=True)
            (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          )
        )
        (glp_1): MLP(
          (linear1): Linear(in_features=128, out_features=64, bias=True)
          (linear2): Linear(in_features=64, out_features=64, bias=True)
          (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((6

In [9]:
model.module

AttributeError: 'TNT' object has no attribute 'module'

In [11]:
# Export the model
torch.onnx.export(model,                                 # model being run
                  data_sample,                   # model input (or a tuple for multiple inputs)
                  "onnx_models/tnt_example.onnx", # where to save the model (can be a file or file-like object)
                  export_params=True,                           # store the trained parameter weights inside the model file
                  opset_version=10,                             # the ONNX version to export the model to
                  do_constant_folding=True,                     # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'],                    # the model's output names
                  
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

/home/techtoker/tools/anaconda3/envs/dl/lib/python3.9/site-packages/torch/onnx/utils.py:366: UserWarning: Skipping _decide_input_format
 getattr(): attribute name must be string
  warnings.warn("Skipping _decide_input_format\n {}".format(e.args[0]))


RuntimeError: Only tuples, lists and Variables are supported as JIT inputs/outputs. Dictionaries and strings are also accepted, but their usage is not recommended. Here, received an input of unsupported type: GraphData